In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import os

PROJECT_ROOT=os.path.join(os.path.dirname(os.path.abspath(os.pardir)))
sys.path.append(PROJECT_ROOT)

from tqdm import tqdm
import torch
import torchvision
from torchvision.transforms import transforms 
import matplotlib.pyplot as plt
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np
from PIL import Image

from datetime import date

from object_detection.mnist_model import MNIST
from object_detection.fgsm_attack import fgsm_attack
from object_detection import mnist_inference
from object_detection import mnist_evaluation
from object_detection.transform import Invertor
from object_detection.mnist_augmentation import AlbuAugmentation
from object_detection.transform import Convertor


In [ ]:
CONFIG = {"batch_size": 200}
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SAVE_DPATH = os.path.join(PROJECT_ROOT,'results','mnist_train_with_aug')
os.makedirs(SAVE_DPATH, exist_ok=True)

In [ ]:
# Загружаем обученную модель
model = MNIST()
model = model.to(DEVICE)

MODEL_FPATH = os.path.join(PROJECT_ROOT, "checkpoints", "mnist_checkpoints", "best_with_aug.pth")
model.load_state_dict(torch.load(MODEL_FPATH)["model_state"])

# Инициализируем инференс 
infer = mnist_inference.Inference(model, device= DEVICE)

# установим модель в режим оценки. В данном случае это относится к слоям отсева
model.eval()


Рассмотрим показатели метрик модели обученной с альбументациями на исходной тестовой выборке  

In [ ]:
# загружаем тестовую выборку
test_data = torchvision.datasets.MNIST(
   os.path.join(PROJECT_ROOT, "mnist_content"), train=False, transform=Invertor(), download=True
)


In [ ]:
mnist_evaluator = mnist_evaluation.MnistEvaluator(infer,test_data)

predictions = mnist_evaluator.evaluate()

_SAVE = True

if _SAVE: 
    predictions.to_csv(
        os.path.join(SAVE_DPATH,f'mnist_pred_{date.today()}.csv'),
        index_label='id'
    )

metrics =mnist_evaluator.classification_report()
print(f'Classification report')
print(metrics)


_SAVE = True

if _SAVE:
    metrics.to_csv(
        os.path.join(SAVE_DPATH,f'classification_report_{date.today()}.csv'),
        index_label='label'
    )


Рассмотрим показатели метрик модели обученной с альбументациями на тестовой выборке с альбументациями 

In [ ]:
transform_aug = transforms.Compose(
            [Invertor(), Convertor(), AlbuAugmentation(), transforms.ToTensor()]
        )
test_data_with_augmentation = torchvision.datasets.MNIST(
   os.path.join(PROJECT_ROOT, "mnist_content"), train=False, transform= transform_aug, download=True
)


test_dataloader_aug = torch.utils.data.DataLoader(
    dataset=test_data_with_augmentation, 
    batch_size=1,
    shuffle=False
)

In [ ]:
mnist_evaluator_aug = mnist_evaluation.MnistEvaluator(infer,test_data_with_augmentation)

_SAVE_PRED= True

predictions_aug = mnist_evaluator_aug.evaluate()

if _SAVE_PRED:
    predictions_aug.to_csv(
        os.path.join(SAVE_DPATH,f'pred_with_aug_{date.today()}.csv'),
        index_label='id'
    )

_SAVE_METRICS=True

metrics_aug =mnist_evaluator_aug.classification_report()

print(f'Classification report with augmentations')
print(metrics_aug)

if _SAVE_METRICS:
    metrics_aug.to_csv(
        os.path.join(SAVE_DPATH,f'classification_report_with_aug_{date.today()}.csv'),
        index_label='label'
    )



Рассмотрим показатели метрик модели обученной с альбументациями на тестовой выборке с атаками

In [ ]:

transform_attack = transforms.Compose(
    [
        Invertor(),
        transforms.ToTensor()
    ]
)
# загружаем тестовую выборку
test_data_for_attack = torchvision.datasets.MNIST(
    "mnist_content", train=False, transform=transform_attack, download=True
)

test_dataloader_for_attack=torch.utils.data.DataLoader(
    dataset=test_data_for_attack, 
    batch_size=1,
    shuffle=False
)

In [ ]:
def data_fgsm_attack(model, device, test_loader, epsilon ):
    output_data=[]
    origin_data=[]
    # Loop over all examples in test set
    for data, target in tqdm(test_loader):
        # Send the data and label to the device
        data, target = data.to(device), target.to(device)
        
        # Set requires_grad attribute of tensor. Important for Attack
        data.requires_grad = True

        # Forward pass the data through the model
        output = model(data.reshape(-1, 28 * 28))
        init_pred = output.max(1, keepdim=True)[1]# get the index of the max log-probability
        
        # If the initial prediction is wrong, dont bother attacking, just move on
        if init_pred.item() != target.item():
            continue

        # Calculate the loss
        loss = F.nll_loss(output, target)

        # Zero all existing gradients
        model.zero_grad()

        # Calculate gradients of model in backward pass
        loss.backward()

        # Collect datagrad
        data_grad = data.grad.data
        
        # Call FGSM Attack
        perturbed_data = fgsm_attack(data.reshape(-1, 28 * 28), epsilon, data_grad.reshape(-1, 28 * 28))

        # Transform tensor to type PIL.Image.Image 
        perturbed_data = np.reshape(perturbed_data.cpu().detach().numpy(),(28,28))
        perturbed_data = Image.fromarray(perturbed_data)

        output_data.append([perturbed_data,target.item()])

        # Collect origin data examples for visualisation 
        original_img = np.reshape(data.cpu().detach().numpy(),(28,28))
        original_img = Image.fromarray(original_img)
        origin_data.append([original_img,target.item()])

    return output_data, origin_data


In [ ]:
_SAVE_PRED= True
_SAVE_METRICS=True
eps=0.05

# Getting data with fgsm attack
print('\n Creating perturbed data ')
perturbed_data,_ = data_fgsm_attack(model,DEVICE,test_dataloader_for_attack,eps)

mnist_evaluator_attack= mnist_evaluation.MnistEvaluator(infer,perturbed_data)
# Getting predictions 
print('Getting predictions')
predictions_attack = mnist_evaluator_attack.evaluate()
# Getting metrics
metrics_attack =mnist_evaluator_attack.classification_report()

print(f'Classification report with fgsm attack eps={eps}')
print(metrics_attack)

if _SAVE_PRED:
    predictions_attack.to_csv(
        os.path.join(SAVE_DPATH,f'pred_eps_{eps}_{date.today()}.csv'),
        index_label='id'
    )
if _SAVE_METRICS:
    metrics_attack.to_csv(
        os.path.join(SAVE_DPATH,f'classification_report_eps_{eps}_{date.today()}.csv'),
        index_label='label'
    )



Рассмотрим показатели метрик модели обученной с альбументациями на тестовой выборке с атаками и с альбуметациями 

In [ ]:
_SAVE_PRED= True
_SAVE_METRICS=True
eps=0.05

# Getting data with fgsm attack
print('\n Creating perturbed data with aug ')
perturbed_data_aug,_ = data_fgsm_attack(model,DEVICE,test_dataloader_aug,eps)

mnist_evaluator_att_aug= mnist_evaluation.MnistEvaluator(infer,perturbed_data_aug)
# Getting predictions 
print('Getting predictions')
predictions_attack_aug = mnist_evaluator_att_aug.evaluate()
# Getting metrics
metrics_attack_aug = mnist_evaluator_att_aug.classification_report()

print(f'Classification report with fgsm attack eps={eps} and aug')
print(metrics_attack_aug)

if _SAVE_PRED:
    predictions_attack_aug.to_csv(
        os.path.join(SAVE_DPATH,f'pred_aug_eps_{eps}_{date.today()}.csv'),
        index_label='id'
    )
if _SAVE_METRICS:
    metrics_attack_aug.to_csv(
        os.path.join(SAVE_DPATH,f'class_report_aug_eps_{eps}_{date.today()}.csv'),
        index_label='label'
    )

